In [105]:
ls

1325Subjects
SST.zip
SST_cleaned_7231.csv
SST_cleaned_7231_1.csv
SST_cleaned_7231_2.csv
SST_cleaned_7231_3.csv
SST_cleaned_7231_all_rows_all_columns.csv
SST_curated.csv
SST_problems.ipynb
SST_quality_control.ipynb
SST_recc_full.csv
Untitled.ipynb
Untitled1.ipynb
accuracy.csv
accuracy_with_avg_ssd.csv
all_subs.csv
concat_SST.ipynb
incomplete_subjects.csv
incorrect_go.csv
partially_curated.csv
reran_partially_cleaned.csv
stop_trials.csv
under_review_mh.csv
weirdcols.csv
zero_ssd.csv


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#this makes it so that the WHOLE dataframe prints, just fyi, this can bog down your code

pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199

In [ ]:
# incomplete_runs_subj = [] 
# SST_concat = pd.DataFrame()
# incomplete_runs_df = pd.DataFrame()
# raw_concat = raw_concat[np.logical_and(raw_concat.TrialCode != 'BeginFix', \
#                                        raw_concat.TrialCode != 'EndFix')]

# raw_concat = raw_concat[~raw_concat['TrialCode'].isnull()]

# for i in raw_concat['NARGUID'].unique(): 
#     sub_df = raw_concat.loc[raw_concat['NARGUID'] == i]
#     #add TrialNum col from 1-360
#     sub_df['TrialNum'] = np.arange(1, len(sub_df)+1)
# #     stop_trial_mask = (sub_df['TrialCode'] == 'IncorrectStop') | \
# #                       (sub_df['TrialCode'] == 'CorrectStop')
# #     stop_trial_idx = stop_trial_mask[stop_trial_mask == True].index
# #     sub_df['StopTrial'] = ""
# #     sub_df['StopTrial'][stop_trial_idx] = np.arange(1, len(sub_df.loc[stop_trial_idx])+ 1)
#     if len(~sub_df['TrialNum'].isnull()) == 360: 
#         if 'StopTooEarly' not in sub_df['TrialCode'].unique(): 
#             SST_concat = SST_concat.append(sub_df)
#     else: 
#         incomplete_runs_df = incomplete_runs_df.append(sub_df)
#         incomplete_runs_subj.append(i)
    

# incomplete_runs_df.to_csv('incomplete_subjects.csv')

In [ ]:
#SST_concat.to_csv('reran_partially_cleaned.csv')

In [ ]:
SST_concat = pd.read_csv('reran_partially_cleaned.csv')

### Check "TrialCode" accuracy

In [ ]:
#how to figure out what response is the correct response 

In [ ]:
#some subjects have their subtrial under Procedure[Trial]
SST_concat['Procedure[SubTrial]'].loc[SST_concat['Procedure[SubTrial]'].isnull()] = SST_concat['Procedure[Trial]'] 
SST_concat['trial_type'] = SST_concat['Procedure[SubTrial]'].replace('VariableStopTrial.*', 'StopTrial', regex=True)

In [ ]:
go_trial_mask = SST_concat.loc[SST_concat['Procedure[SubTrial]'] == 'GoTrial']
go_trial_idx = go_trial_mask[go_trial_mask==True].index

In [ ]:
SST_concat['Go.CRESP'].unique()

In [ ]:
#the response recordings are inconsistent in type and format
cresp_replace = {'2.0': 2.0,
                 '1.0': 1.0,
                 '3.0': 3.0,
                 '4.0': 4.0,
                 '1,{LEFTARROW}': 1.0,
                 '2,{RIGHTARROW}': 2.0}

resp_replace = {'2.0': 2.0,
                '1.0': 1.0,
                '3.0': 3.0,
                '4.0': 4.0,
                '{LEFTARROW}': 1.0,
                '{RIGHTARROW}': 2.0}

SST_concat['Go.RESP'].replace(to_replace = resp_replace, inplace = True)
SST_concat['Go.CRESP'].replace(to_replace = cresp_replace, inplace=True)

In [ ]:
#calc my own correct
SST_concat['correct_go_response'] = np.NaN

SST_concat['correct_go_response'].loc[(~SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] == SST_concat['Go.RESP'])] = float(1)

SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] == SST_concat['Fix.RESP'])] = float(1)

SST_concat['correct_go_response'].loc[(~SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] != SST_concat['Go.RESP']) &
                                      (SST_concat['trial_type'] == 'GoTrial')] = float(0)


SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & 
                                      (SST_concat['Go.CRESP'] != SST_concat['Fix.RESP']) &
                                      (SST_concat['trial_type'] == 'GoTrial')] = float(0)


SST_concat['correct_go_response'].loc[(SST_concat['Go.RESP'].isnull()) & (SST_concat['Fix.RESP'].isnull()) & 
                                      (SST_concat['trial_type'] == 'GoTrial')] = 'omission'




In [ ]:
SST_concat.loc[SST_concat['trial_type'] == 'GoTrial']['correct_go_response'].unique() 

In [ ]:
SST_concat['correct_stop'] = np.NaN

SST_concat['correct_stop'].loc[(SST_concat['StopSignal.RESP'].isnull()) & (SST_concat['Fix.RESP'].isnull()) & \
                               (SST_concat['trial_type'] == 'StopTrial') & (SST_concat['SSD.RESP'].isnull())] = float(1)

SST_concat['correct_stop'].loc[(~(SST_concat['StopSignal.RESP'].isnull()) | ~(SST_concat['Fix.RESP'].isnull()) \
                                | ~(SST_concat['SSD.RESP'].isnull())) & (SST_concat['trial_type'] == 'StopTrial')] = float(0)

In [ ]:
SST_concat['correct_stimulus_mapping_1'] = np.NaN
SST_concat['correct_stimulus_mapping_2'] = np.NaN

In [ ]:
SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp', 'correct_stimulus_mapping_1'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp']['Go.CRESP'].dropna().unique()[0]
SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp', 'correct_stimulus_mapping_2'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Right_Arrow.bmp']['Go.CRESP'].dropna().unique()[1]

SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp', 'correct_stimulus_mapping_1'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp']['Go.CRESP'].dropna().unique()[0]
SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp', 'correct_stimulus_mapping_2'] = SST_concat.loc[SST_concat['Stimulus'] == 'images/Left_Arrow.bmp']['Go.CRESP'].dropna().unique()[1]

In [ ]:
SST_concat['correct_stop_mapping'] = np.NaN #work on this 
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (~SST_concat['SSD.RESP'].isnull()) &\
                                       ((SST_concat['SSD.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['SSD.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (~SST_concat['SSD.RESP'].isnull()) &\
                                      (SST_concat['SSD.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['SSD.RESP'] != SST_concat['correct_stimulus_mapping_2'])] = float(0)


In [ ]:
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (~SST_concat['StopSignal.RESP'].isnull()) &\
                                       ((SST_concat['StopSignal.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['StopSignal.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (~SST_concat['StopSignal.RESP'].isnull()) &\
                                       ((SST_concat['StopSignal.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['StopSignal.RESP'] != SST_concat['correct_stimulus_mapping_2']))] = float(0)

In [ ]:
SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (SST_concat['StopSignal.RESP'].isnull()) & (~SST_concat['Fix.RESP'].isnull()) &\
                                       ((SST_concat['Fix.RESP'] == SST_concat['correct_stimulus_mapping_1']) | (SST_concat['Fix.RESP'] == SST_concat['correct_stimulus_mapping_2']))] = float(1)

SST_concat['correct_stop_mapping'].loc[(SST_concat['correct_stop'] == 0) & (SST_concat['SSD.RESP'].isnull()) & (SST_concat['StopSignal.RESP'].isnull()) & (~SST_concat['Fix.RESP'].isnull()) &\
                                       ((SST_concat['Fix.RESP'] != SST_concat['correct_stimulus_mapping_1']) & (SST_concat['Fix.RESP'] != SST_concat['correct_stimulus_mapping_2']))] = float(0)

### fix the go.rt calculation

In [ ]:
#find the index of where there's a fixation response on go trials 
go_fix_resp = (~SST_concat['Fix.RESP'].isnull()) & \
              (SST_concat['Go.RESP'].isnull()) & \
              (SST_concat['trial_type'] == 'GoTrial')
go_fix_idx = go_fix_resp[go_fix_resp == True].index

In [ ]:
#make column from Go.RT values
SST_concat['go_rt_adjusted'] = SST_concat['Go.RT'].copy()
#for long response trials, add the fix.rt to the go.duration
SST_concat['go_rt_adjusted'][go_fix_idx] = SST_concat.loc[go_fix_idx]['Go.Duration'] +  \
                                           SST_concat.loc[go_fix_idx]['Fix.RT']

#### fix the stop sig rt calculation

In [ ]:
#find all stop trials
stop_trial_mask = (SST_concat['trial_type'] == 'StopTrial')
#make an index of all stop trials 
stop_trial_idx = stop_trial_mask[stop_trial_mask == True].index

In [ ]:
SST_concat['stop_rt_adjusted'] = SST_concat['StopSignal.RT']
#find all stop failure trials with resp during fix.resp, and no resp during stopsignal.resp
stop_fix_resp = (~SST_concat['Fix.RESP'].isnull()) & \
                (SST_concat['StopSignal.RESP'].isnull()) & \
                ((SST_concat['trial_type'] == 'StopTrial') & SST_concat['correct_stop'] == 0)

stop_fix_idx = stop_fix_resp[stop_fix_resp == True].index

stop_SSD_resp = (~SST_concat['SSD.RESP'].isnull()) & \
                (~(SST_concat['StopSignal.RESP'].isnull()) | ~(SST_concat['Fix.RESP'].isnull()) \
                    | ~(SST_concat['SSD.RESP'].isnull()))

In [ ]:
#use that index to add stop signal duration to get correct stop signal RT on these trials
SST_concat['stop_rt_adjusted'][stop_fix_resp] = SST_concat.loc[stop_fix_resp]['StopSignal.Duration'] +\
                                           SST_concat.loc[stop_fix_resp]['Fix.RT']

SST_concat['stop_rt_adjusted'][stop_SSD_resp] = SST_concat.loc[stop_SSD_resp]['SSD.RT']

In [ ]:
#find the stop failure trials and add the stop signal duration for correct stop fail RT
stop_resp_mask = ((~(SST_concat['StopSignal.RESP'].isnull()) | ~(SST_concat['Fix.RESP'].isnull()) \
                                | ~(SST_concat['SSD.RESP'].isnull())) & \
                (~SST_concat['stop_rt_adjusted'].isnull()))
        
stop_resp_idx = stop_resp_mask[stop_resp_mask == True].index

SST_concat['stop_rt_adjusted'][stop_resp_idx] = \
        SST_concat['stop_rt_adjusted'][stop_resp_idx] + SST_concat['SSDDur'][stop_resp_idx]

In [ ]:
SST_concat.loc[(SST_concat['TrialCode'] == 'CorrectStop') & (SST_concat['Fix.RT'] != 0)]\
        [['SSD.RT', 'StopSignal.RT', 'Fix.RT', 'correct_stop']]

potential figure 

In [ ]:
stop_rt_adjuse

In [ ]:
#this is still unresolved
plt.hist(SST_concat.loc[(SST_concat['TrialCode'] == 'CorrectStop') & (SST_concat['stop_rt_adjusted'] != 0)]['stop_rt_adjusted'], bins = 20)
plt.show()
len(SST_concat.loc[(SST_concat['TrialCode'] == 'CorrectStop') & (SST_concat['stop_rt_adjusted'] != 0)])

### create go stimuli duration

In [100]:
#create column for Go.Stim.Duration - when there's no go response, this should be 1000ms
SST_concat['go_stim_duration'] = SST_concat['Go.RT'].copy()
SST_concat['go_stim_duration'].loc[SST_concat['Go.RT'] == 0] = SST_concat['Go.Duration'].loc[SST_concat['Go.RT'] == 0]

SST_concat['go_stim_duration'].loc[~(SST_concat['SSD.RESP'].isnull())] = SST_concat.loc[~SST_concat['SSD.RESP'].isnull()]['SSD.RT']
SST_concat['go_stim_duration'].loc[(SST_concat['SSD.RT']  == 0)] = SST_concat.loc[SST_concat['SSD.RT'] == 0]['SSDDur']


/Users/work/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


#### TESTING

In [ ]:
# SST_concat.loc[(SST_concat['correct_go_response'] !=0) & (SST_concat['TrialCode'] == 'IncorrectGo')] \
#   [['TrialCode', 'Go.RT', 'Go.CRESP', 'Go.RESP', 'Fix.RESP', 'Fix.RT', 'TrialNum']]

In [ ]:
# i=0
# for n in nar: 
#     print(n, i)
    
#     initial_ssd = SST_concat.loc[(SST_concat['NARGUID'] == n) & (SST_concat['StopTrial'] == stop.iloc[i])]['SSDDur']
#     print(SST_concat.loc[(SST_concat['NARGUID'] == n) & (SST_concat['StopTrial'] == stop.iloc[i])][['StopTrial', 'correct_stop', 'TrialCode']], initial_ssd)

#     post_ssd = SST_concat.loc[(SST_concat['NARGUID'] == n) & (SST_concat['StopTrial'] == stop_plus.iloc[i])]['SSDDur']
#     print(SST_concat.loc[(SST_concat['NARGUID'] == n) & (SST_concat['StopTrial'] == stop_plus.iloc[i])][['StopTrial', 'correct_stop', 'TrialCode']], post_ssd)

#  #   print(i, initial_ssd, post_ssd)
#     i+=1

In [ ]:
# SST_concat.loc[(SST_concat['correct_go_response'] ==0) & (SST_concat['TrialCode'] == 'CorrectGo') & (SST_concat['Fix.RESP'] == SST_concat['Go.CRESP'])][:1000] \
#     [['Go.CRESP', 'Go.RESP', 'Go.RT', 'Fix.RESP', 'Fix.RT']]


In [ ]:
len(SST_concat.loc[(SST_concat['correct_stop'] ==0) & (SST_concat['TrialCode'] == 'CorrectStop')] \
    [['Fix.RESP','Fix.RT' ,'StopSignal.RT', 'StopSignal.RESP', 'SSD.RT', 'SSD.RESP',  'SSDDur']])
    

In [ ]:
len(SST_concat.loc[(SST_concat['correct_stop'] ==1) & (SST_concat['TrialCode'] == 'IncorrectStop')])

In [ ]:
SST_concat.loc[SST_concat['trial_type'] =='GoTrial']['TrialCode'].unique()

In [ ]:
SST_concat.loc[SST_concat['trial_type'] =='StopTrial']['TrialCode'].unique()

In [ ]:
#fix_minus_go = SST_concat.loc[go_trial_idx]['Fix.OnsetTime'] - SST_concat.loc[go_trial_idx]['Go.OnsetTime']


In [ ]:
#SST_concat.iloc[:867600].to_csv('SST_cleaned_7231_1.csv')

In [ ]:
#SST_concat.iloc[867600:1735560, :].to_csv('SST_cleaned_7231_2.csv')

In [ ]:
#SST_concat.iloc[1735560:].to_csv('SST_cleaned_7231_3.csv')

In [104]:
SST_concat.to_csv('SST_cleaned_7231_all_rows_all_columns.csv')

In [103]:
len(SST_concat)

2603160